# Dualモデル学習用
- 後でリファクタリング

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
from numba import jit
from tqdm import tqdm

%matplotlib inline

現状GPUメモリに乗りきらずCPUで回している<br>
VGGの出力をあらかじめpickleとして保存することで回避可能か？

## Model Definition

In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'
import keras

/home/pip/koki_ishizuka/.conda/envs/py35-zukapy/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

In [4]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input,Layer,Lambda
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dense,Dropout
from keras.layers import concatenate
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K

In [5]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

#### ハイパーパラメータ
- `alpha > beta`を満たす必要がある(元論文)
- dense_numで特徴が潰れないように
    - dualだから100にしてた(Singleを200次元でやったので100+100で200になるから平等性)
    - 200+200だと多すぎる？

In [6]:
imheight = 128
imwidth = 128
channels = 3
# ALPHA = 0.2
ALPHA=0.1
BETA=0.05
dense_num = 150
vec_length=dense_num*2

#### VGG16読み込み

In [7]:
from keras.applications.vgg16 import VGG16
#include_top=false => Dense不要
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(imwidth, imheight, channels)), input_shape=None) 

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
for layer in base_model.layers[:15]:
    layer.trainable=False

#### 各ルートのCNNを共通で定義


In [9]:
# shallow network for colorized features
# create_embNetの中でやるべき？
inputs = Input(shape=(imwidth, imheight, channels))
conv1 = Conv2D(32, (4,4) , padding='same', activation='relu')(inputs)
pool1 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(conv1)
conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(conv2)
flatten = Flatten()(pool2) 
dense_layer = Dense(dense_num, activation='relu')(flatten)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer1')(dense_layer)
shallow_model=Model(inputs=inputs,outputs=norm_layer) 

In [10]:
def create_embNet():
    shallow_inputs = Input(shape=(imwidth, imheight, channels))
    x = base_model.output
    conv1 = Conv2D(32, (4,4) , padding='same', activation='relu')(x)
    conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(conv1)
    flatten = Flatten()(conv2) 
    dense_layer = Dense(dense_num, activation='relu')(flatten)
    norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)
    # inputに対してshallow_modelのoutputも用意
    x1 = norm_layer
    x2 = shallow_model(shallow_inputs)
    out = concatenate([norm_layer,x2])
    return Model(inputs=[base_model.input,shallow_inputs],outputs=out)
#     return  Model(inputs=[base_model.input], outputs=norm_layer)

In [11]:
# define three Inputs
# create embNetの中でInputを定義すると明示的に3つの入力が分けられない
a_in = Input(shape = (imheight, imwidth, channels), name='anchor_input')
p_in = Input(shape = (imheight, imwidth, channels), name='positive_input')
n_in = Input(shape = (imheight, imwidth, channels), name='negative_input')

In [12]:
sa_in = Input(shape = (imheight, imwidth, channels), name='sanchor_input')

In [13]:
# 学習後にベクトルのencoderとして利用するので外に定義する。
con_embNet = create_embNet()
shop_embNet = create_embNet()

In [14]:
a_emb = shop_embNet([a_in,a_in])
p_emb = con_embNet([p_in,p_in])
n_emb = con_embNet([n_in,n_in])

### Triplet loss
- 実行時コメントアウトはずす

In [15]:
class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss
    
    def get_config(self):
        config = {'alpha': self.alpha}
        base_config = super(TripletLossLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [16]:
# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=ALPHA, name='triplet_loss_layer')([a_emb, p_emb, n_emb])

# Model that can be trained with anchor, positive negative images
tripletNet = Model([a_in, p_in, n_in], triplet_loss_layer)
tripletNet.compile(loss=None, optimizer='adam')

### Improved Triplet Loss
- `Loss=[D(a,p)-D(a,n)+ALPHA]+[D(a,p)-BETA]`
- Positiveを短くする方向に制御する

https://qiita.com/tancoro/items/35d0925de74f21bfff14#improved-triplet-loss

<img src="./readme_imgs/improved.PNG" width=50% align=left><br>



In [15]:
class TripletLossLayer(Layer):
    def __init__(self, alpha, beta, **kwargs):
        self.alpha = alpha
        self.beta = beta
        super(TripletLossLayer, self).__init__(**kwargs)
    #a-pとa-nの距離差がmarginになるように
#     in-class variance(Positive Variance)を抑制する項を追加
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        pn_dist = K.sum(K.square(p-n), axis=-1)
        return K.sum(K.maximum((p_dist - n_dist + self.alpha), 0) + K.maximum((p_dist - self.beta), 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss
    
    def get_config(self):
        config = {'alpha': self.alpha}
        base_config = super(TripletLossLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [16]:
# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=ALPHA, beta=BETA, name='triplet_loss_layer')([a_emb, p_emb, n_emb])

# Model that can be trained with anchor, positive negative images
tripletNet = Model([a_in, p_in, n_in], triplet_loss_layer)

In [17]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(tripletNet, show_shapes=True, show_layer_names=False).create(prog='dot', format='svg'))

In [18]:
#並列化
# from keras.utils import multi_gpu_model
# tripletNet2 = multi_gpu_model(tripletNet,gpus=2)
# compile model
tripletNet.compile(loss=None, optimizer='adam')

### データの用意

In [19]:
import os

BASE_PATH = './dataset/T_Shirt_all/'
ids = sorted([x for x in os.listdir(BASE_PATH)])

In [20]:
ids[0]

'id_00000001'

In [21]:
len(ids)

6155

### tripletのパスの組を返す関数

In [22]:
import itertools
import random

# @jit
def get_triplets(ids,BASE_PATH):
    triplets=[]
    # idの選択の仕方をランダムにしたらいいのか？
    for id_ in tqdm(ids):
        files = sorted([BASE_PATH+id_+'/'+x for x in os.listdir(BASE_PATH+id_)])
#         print(files)
        con = sorted([x for x in files if 'comsumer' in x])
        shop = sorted([x for x in files if 'shop' in x ])
        combs = list(itertools.product(tuple(con),tuple(shop)))
        
        for comb in combs:
#             print(len(comb))
            comb = list(comb)
            neg_id = random.choice([x for x in ids if x != id_])
#             print(neg_id)
#             print(len([BASE_PATH+neg_id+'/'+x for x in os.listdir(BASE_PATH+neg_id) if 'shop' in x]))
            neg_file = random.choice([BASE_PATH+neg_id+'/'+x for x in os.listdir(BASE_PATH+neg_id) if 'shop' in x])
            comb.append(neg_file)
            triplets.append(comb)
    
    return triplets

### id単位でtrainとtestを分割する
- 元々np.random.choice()でやっていたが、ブートストラップサンプリングだったのでダメ
- train_test_splitを利用

In [23]:
from sklearn.cross_validation import train_test_split
# from sklearn.model_selection import ShuffleSplit

/home/pip/koki_ishizuka/.conda/envs/py35-zukapy/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
# random_stateを固定しておく
train_ids,test_ids=train_test_split(ids,test_size=0.33,random_state=0)
# idsは今後使わないので削除
del ids

In [25]:
train_ids[:5]

['id_00016780', 'id_00007427', 'id_00029554', 'id_00020254', 'id_00018517']

### Triplet作成用関数
- エポックごとにTripletの組み合わせをランダムに変更するためpickle保存ができない
- エポックごとに変えるのは普通？

In [26]:
from PIL import Image
# @jit
def get_np_triplets(triplet_PATHs):
    triplets = []
    # triplets = np.ndarray
    for triplet in tqdm(triplet_PATHs):

#         anc_img = Image.fromarray(np.uint8(triplet[0])).convert('RGB')
#         pos_img = Image.fromarray(np.uint8(triplet[1])).convert('RGB')
#         neg_img = Image.fromarray(np.uint8(triplet[2])).convert('RGB')

        anc_img = Image.open(triplet[0]).convert('RGB')
        pos_img = Image.open(triplet[1]).convert('RGB')
        neg_img = Image.open(triplet[2]).convert('RGB')

        anc_img = np.array(anc_img.resize((128,128)))/255. #resize to (128,128,3)
        pos_img = np.array(pos_img.resize((128,128)))/255.    
        neg_img = np.array(neg_img.resize((128,128)))/255.    

        tri = [anc_img,pos_img,neg_img]
        triplets.append(np.array(tri))

    triplets = np.array(triplets)
    return triplets

test_idsはretrival.ipynbで参照するのでpickleとして保存

In [27]:
import pickle
f = open('test_ids.txt', 'wb')
pickle.dump(test_ids, f)

# 学習

### 仕様について
- 各エポックでtestデータを用いてN-top accを出したい
    - 学習過程のデータとしては有意義だが時間がかかる
    - 最終的なN-topしか載せていない論文がほとんど
- epochの外でtrain,testに分割するパターンなので常にtestのidは同じ
- train_tripletのnegativeが毎回ランダムになるので偏らないメリットがある
- model.fitはepochs=1で行う

### やるべき
- epochごとにlossをファイル出力

In [28]:
epochs = 120

In [29]:
model_history = []
for epoch in range(epochs):
    print('epoch %s'% epoch)
    if epoch % 5 == 0:
        # PATHの組みを取得
        if epoch != 0: del triplets
        triplets_train_PATHs = get_triplets(train_ids,BASE_PATH)
        # np配列に変換
        triplets = get_np_triplets(triplets_train_PATHs)
        # 使い終わったので削除
        del triplets_train_PATHs
    # fit
    hist = tripletNet.fit([triplets[:,0],triplets[:,1],triplets[:,2]], epochs=1, batch_size=50) # using batch_size is better
    model_history.append(hist.history)
    # 使い終わったので削除
##    del triplets
#     if (epoch+1) % 5 == 0:
#         5epochごとにmodelを保存
#         shop_embNet.save('./model/T_Shirt/improved_tripletloss/a{}b{}/{}/shop_e{}.h5'.format(ALPHA,BETA,vec_length,epoch))
#         con_embNet.save('./model/T_Shirt/improved_tripletloss/a{}b{}/{}/con_e{}.h5'.format(ALPHA,BETA,vec_length,epoch))
#         shop_embNet.save('./model/T_Shirt/tripletloss/a{}/{}/shop_e{}.h5'.format(ALPHA,vec_length,epoch))
#         con_embNet.save('./model/T_Shirt/tripletloss/a{}/{}/con_e{}.h5'.format(ALPHA,vec_length,epoch))

# 学習のhistoryを保存
f = open('./model/T_Shirt/improved_tripletloss/a{}b{}/{}/history.txt'.format(ALPHA,BETA,vec_length),'wb')
pickle.dump(model_history, f)

  0%|          | 19/4123 [00:00<00:34, 117.63it/s]

epoch 0


100%|██████████| 38378/38378 [04:03<00:00, 157.77it/s]


Instructions for updating:
Use tf.cast instead.
Epoch 1/1
38378/38378 [==============================] - 254s 7ms/step - loss: 4.2106
epoch 1
Epoch 1/1
38378/38378 [==============================] - 241s 6ms/step - loss: 3.2381
epoch 2
Epoch 1/1
38378/38378 [==============================] - 242s 6ms/step - loss: 2.3635
epoch 3
Epoch 1/1
38378/38378 [==============================] - 242s 6ms/step - loss: 1.8280
epoch 4
Epoch 1/1
38378/38378 [==============================] - 240s 6ms/step - loss: 1.4437
epoch 5


100%|██████████| 38378/38378 [04:17<00:00, 148.98it/s]


...

Epoch 1/1
38378/38378 [==============================] - 266s 7ms/step - loss: 0.0532
epoch 111
Epoch 1/1
38378/38378 [==============================] - 242s 6ms/step - loss: 0.0122
epoch 112
Epoch 1/1
38378/38378 [==============================] - 246s 6ms/step - loss: 0.0056
epoch 113
Epoch 1/1
38378/38378 [==============================] - 273s 7ms/step - loss: 0.0062
epoch 114
Epoch 1/1
38378/38378 [==============================] - 269s 7ms/step - loss: 0.0062


  0%|          | 0/4123 [00:00<?, ?it/s]

epoch 115


100%|██████████| 38378/38378 [03:56<00:00, 162.16it/s]


Epoch 1/1
38378/38378 [==============================] - 272s 7ms/step - loss: 0.0545
epoch 116
Epoch 1/1
38378/38378 [==============================] - 269s 7ms/step - loss: 0.0120
epoch 117
Epoch 1/1
38378/38378 [==============================] - 271s 7ms/step - loss: 0.0066
epoch 118
Epoch 1/1
38378/38378 [==============================] - 250s 7ms/step - loss: 0.0059
epoch 119
Epoch 1/1
38378/38378 [==============================] - 257s 7ms/step - loss: 0.0069


In [30]:
len(model_history)

120

In [31]:
# 学習のhistoryを保存
f = open('./model/T_Shirt/improved_tripletloss/a{}b{}/{}/history.txt'.format(ALPHA,BETA,vec_length),'wb')
pickle.dump(model_history, f)

# updates
- ver 0.1
    - 学習できることを確認
    - 1epochごとにモデルを保存できるように変更
- ver 0.2
    - embNetを並列化
- ver 0.3(2019/6/30)
    - ~~Triplet lossにshop画像間の距離を考慮するため内積を加える~~
- 2019/10/11
    - Improved Triplet lossを導入
- 2019/11/17
    - Improved Triplet lossを修正\
- 2019/11/22
    - vec_length=512

In [ ]:
aaa = get_triplets(train_ids[:5],BASE_PATH)
print(aaa)
bbb = get_np_triplets(aaa)

for b in bbb:
#     print(b)
    fig = plt.figure(figsize=(8,4))

    plt.subplot(1,3,1)
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.tick_params(color='white')
    plt.imshow(b[0])
    plt.title('anchor')
    
    plt.subplot(1,3,2)
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.tick_params(color='white')    

    plt.imshow(b[1])
    plt.title('anchor')
    plt.subplot(1,3,3)
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.tick_params(color='white')
    plt.imshow(b[2])
    plt.title('anchor')    
    